In [3]:
import pandas as pd

# Load CSV
df = pd.read_csv("./US Airline Flight Routes and Fares 1993-2024.csv")

# Basic check
print(df.shape)
print(df.columns)


C:\Users\user\AppData\Local\Temp\ipykernel_38868\3500053775.py:4: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./US Airline Flight Routes and Fares 1993-2024.csv")


(245955, 23)
Index(['tbl', 'Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2',
       'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'lf_ms', 'fare_low', 'Geocoded_City1', 'Geocoded_City2',
       'tbl1apk'],
      dtype='object')


In [4]:
# Combine route identifier
df["route"] = df["city1"] + "-" + df["city2"]

# Count number of quarters available per route
route_counts = df.groupby("route")["quarter"].count()

# Filter routes with >= 8 quarters of data
valid_routes = route_counts[route_counts >= 8].index

df_filtered = df[df["route"].isin(valid_routes)].copy()

print(f"Routes before filtering: {df['route'].nunique()}")
print(f"Routes after filtering (>=8 quarters): {df_filtered['route'].nunique()}")


Routes before filtering: 1266
Routes after filtering (>=8 quarters): 951


In [5]:
print(df_filtered.isna().sum())  # Missing values

# Quick describe to see fare/passenger distribution
print(df_filtered[["fare", "passengers"]].describe())


tbl                   0
Year                  0
quarter               0
citymarketid_1        0
citymarketid_2        0
city1                 0
city2                 0
airportid_1           0
airportid_2           0
airport_1             0
airport_2             0
nsmiles               0
passengers            0
fare                  0
carrier_lg         1522
large_ms           1522
fare_lg            1522
carrier_low        1594
lf_ms              1594
fare_low           1594
Geocoded_City1    39028
Geocoded_City2    39028
tbl1apk               0
route                 0
dtype: int64
                fare     passengers
count  244950.000000  244950.000000
mean      219.163503     299.625752
std        82.376085     511.950864
min        50.000000       0.000000
25%       164.790000      21.000000
50%       209.490000     112.000000
75%       263.040000     339.000000
max      3377.000000    8301.000000


In [6]:
# Compute route-level aggregates
route_summary = df_filtered.groupby("route").agg({
    "nsmiles": "mean",
    "passengers": "mean"
}).reset_index()

# Sort by distance and passengers to sample diverse routes
short_routes = route_summary.sort_values("nsmiles").head(3)
long_routes = route_summary.sort_values("nsmiles", ascending=False).head(3)
high_passenger = route_summary.sort_values("passengers", ascending=False).head(2)
low_passenger = route_summary.sort_values("passengers").head(2)

# Combine and drop duplicates
selected_routes = pd.concat([short_routes, long_routes, high_passenger, low_passenger]).drop_duplicates("route")

print("Selected Representative Routes:")
print(selected_routes)


Selected Representative Routes:
                                                 route      nsmiles  \
554  Hartford, CT-New York City, NY (Metropolitan A...   115.000000   
709  Los Angeles, CA (Metropolitan Area)-San Diego, CA   119.989691   
808  New York City, NY (Metropolitan Area)-Philadel...   130.000000   
766          Miami, FL (Metropolitan Area)-Seattle, WA  2724.000000   
150  Boston, MA (Metropolitan Area)-San Francisco, ...  2704.000000   
757         Miami, FL (Metropolitan Area)-Portland, OR  2700.000000   
439                             Denver, CO-Phoenix, AZ   602.000000   
878                            Phoenix, AZ-Seattle, WA  1107.985507   
828  New York City, NY (Metropolitan Area)-Steamboa...  1782.000000   
540  Greenville/Spartanburg, SC-Los Angeles, CA (Me...  2057.000000   

      passengers  
554    32.875000  
709    67.938144  
808    28.761194  
766   294.775424  
150   337.309793  
757   119.460870  
439  2615.857143  
878  2217.231884  
828    14.11538

In [7]:
df_sample = df_filtered[df_filtered["route"].isin(selected_routes["route"])]
df_sample.to_csv("sample_routes.csv", index=False)

print("Saved 5–10 representative routes to sample_routes.csv")


Saved 5–10 representative routes to sample_routes.csv


In [8]:
new_df = pd.read_csv("./sample_routes.csv")
new_df

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk,route
0,Table1a,2021,3,30325,30466,"Denver, CO","Phoenix, AZ",11292,14107,DEN,...,WN,0.5098,133.45,AA,0.2062,121.14,NaN,NaN,202131129214107DENPHX,"Denver, CO-Phoenix, AZ"
1,Table1a,2021,3,30466,30559,"Phoenix, AZ","Seattle, WA",14107,14747,PHX,...,AS,0.4525,130.35,WN,0.2302,116.76,NaN,NaN,202131410714747PHXSEA,"Phoenix, AZ-Seattle, WA"
2,Table1a,2021,3,30721,32457,"Boston, MA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",10721,13796,BOS,...,WN,0.5494,319.14,WN,0.5494,319.14,NaN,NaN,202131072113796BOSOAK,"Boston, MA (Metropolitan Area)-San Francisco, ..."
3,Table1a,2021,3,30721,32457,"Boston, MA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",10721,14771,BOS,...,B6,0.4795,419.18,UA,0.3009,411.61,NaN,NaN,202131072114771BOSSFO,"Boston, MA (Metropolitan Area)-San Francisco, ..."
4,Table1a,2021,3,30721,32457,"Boston, MA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",10721,14831,BOS,...,B6,0.6287,317.16,DL,0.1461,288.81,NaN,NaN,202131072114831BOSSJC,"Boston, MA (Metropolitan Area)-San Francisco, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,Table1a,2024,1,30721,32457,"Boston, MA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",14307,14831,PVD,...,DL,0.5769,490.30,WN,0.3077,217.61,NaN,NaN,202411430714831PVDSJC,"Boston, MA (Metropolitan Area)-San Francisco, ..."
1869,Table1a,2024,1,32467,30559,"Miami, FL (Metropolitan Area)","Seattle, WA",11697,14747,FLL,...,AS,0.5326,286.91,AS,0.5326,286.91,NaN,NaN,202411169714747FLLSEA,"Miami, FL (Metropolitan Area)-Seattle, WA"
1870,Table1a,2024,1,32467,30559,"Miami, FL (Metropolitan Area)","Seattle, WA",13303,14747,MIA,...,AS,0.4662,332.52,AS,0.4662,332.52,NaN,NaN,202411330314747MIASEA,"Miami, FL (Metropolitan Area)-Seattle, WA"
1871,Table1a,2024,1,32467,34057,"Miami, FL (Metropolitan Area)","Portland, OR",11697,14057,FLL,...,AS,0.5744,252.02,AS,0.5744,252.02,NaN,NaN,202411169714057FLLPDX,"Miami, FL (Metropolitan Area)-Portland, OR"
